In [1]:
# pip install --upgrade kaggle-environments

In [1]:
# # imports
# !pip install stable-baselines3 pyglet imageio-ffmpeg gymnasium[atari,accept-rom-license] ale-py > /dev/null 2>&1
!pip install gymnasium==1.0.0
!pip install ale_py
!pip install stable-baselines3
# !pip install stable-baselines3[extra]
# pip install sb3-contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 17.7 MB/s eta 0:00:0000:01
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.9 requires gymnasium==0.29.0, but you have gymnasium 1.0.0 which is incompatible.
stable-baselines3 2.1.0 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 19.9 MB/s eta 0:00:0000:01
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0
    Uninstalling gymnasium-1.0.0:
      Successfully uninstalled gymnasium-1.0.0
ERROR: pip's depen

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import ale_py

In [7]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import torch
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback

from gymnasium.wrappers import ResizeObservation

In [8]:
print(gym.__version__)

0.29.1


In [9]:
from stable_baselines3.common.monitor import Monitor

In [10]:
import matplotlib.pyplot as plt

In [11]:
import os

In [12]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback

In [13]:
gym.register_envs(ale_py)

In [12]:
# from sb3_contrib import QRDQN

In [14]:
from datetime import datetime
from stable_baselines3 import A2C
from stable_baselines3.ppo.policies import MlpPolicy
from wandb.integration.sb3 import WandbCallback

In [15]:
from stable_baselines3.common.env_util import make_vec_env

In [16]:
# ENV_NAME = 'ALE/MarioBros-v5'

# configuration file
config = {
    "policy_type": "CnnPolicy",
    "total_timesteps": 10000000, # 1000000
    "env_name": "ALE/Breakout-v5",
    "model_name": "Breakout-v5",
    "export_path": "/kaggle/working/exports/",
    "videos_path": "/kaggle/working/videos/",
}



In [17]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY ")

In [18]:
# Wandb setup
wandb.login(key=secret_value_0)
run = wandb.init(
    project="Project_1",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    save_code=True,  # optional
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amrit0081 (education_ai). Use `wandb login --relogin` to force relogin


In [17]:

class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super().__init__(env)
        # Check that 'FIRE' is a valid action in the environment
        assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
        assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

    def step(self, action):
        return self.env.step(action)

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)

        # Perform the FIRE action
        obs, _, terminated, truncated, _ = self.env.step(1)
        if terminated or truncated:  # If game ends after FIRE, reset again
            obs, info = self.env.reset(**kwargs)

        return obs, info
        
# Custom wrapper to add channel dimension
class AddChannelDimension(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape
        # Update the observation space to include a channel dimension
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(obs_shape[0], obs_shape[1], 1),
            dtype=np.uint8,
        )

    def observation(self, observation):
        # Add a channel dimension
        return np.expand_dims(observation, axis=-1)
        

def make_env(env_name, obs_type="grayscale", render_mode=None,):
    def _init():
        env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
        print("Standard Env.        : {}".format(env.observation_space.shape))
        env = FireResetEnv(env)
        print("FireResetEnv          : {}".format(env.observation_space.shape))
        env = ResizeObservation(env, (84, 84))
        print("ResizeObservation    : {}".format(env.observation_space.shape))
        env = AddChannelDimension(env)  # Add channel dimension here
        print("AddChannelDimension  : {}".format(env.observation_space.shape))
        # env = GrayscaleObservation(env, keep_dim=True)
        # print("GrayscaleObservation : {}".format(env.observation_space.shape))
        env = ScaledFloatFrame(env)
        print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        env = Monitor(env, allow_early_resets=False) # from stable baselines
        print("Monitor               : {}".format(env.observation_space.shape))

        return env
    return _init



# env = DummyVecEnv([make_env(config["env_name"])])
# use make_vec_env with n_envs=16
env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=50)

# stack 4 frames
env = VecFrameStack(env, n_stack=4)
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# convert back to PyTorch format (channel-first)
env = VecTransposeImage(env)
print("Final Observation Space: {}".format(env.observation_space.shape))

A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]


Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
M

In [18]:
# define
model = A2C(config["policy_type"], env, verbose=0, tensorboard_log=f"runs/{run.id}", device="cuda")


In [ ]:
# train
t0 = datetime.now()
model.learn(total_timesteps=config["total_timesteps"], callback=WandbCallback(verbose=2))
t1 = datetime.now()
print('>>> Training time (hh:mm:ss.ms): {}'.format(t1-t0))

# save and export model
model.save(config["export_path"] + config["model_name"])

Training time without parallezing env: >>> Training time (hh:mm:ss.ms): 1:01:58.577332

In [ ]:
print("Finish")

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/exports")
# os.remove("/kaggle/working/Breakout-v5.zip")

In [ ]:
# !zip -r folder_name.zip exports
# from google.colab import files
# files.download('folder_name.zip')